In [58]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [59]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

In [60]:
# Navigate to the website
url = "https://www.construction-europe.com/search?keyword=Caterpillar"
driver.get(url)
time.sleep(2)

In [61]:
# Click the cookie button
driver.find_element(By.CSS_SELECTOR, '#cookieConsent > div > div > div > div.col-12.col-md-auto.text-center.py-2.align-self-center > button').click()

In [62]:
req = urllib.request.Request(url)
sourcecode = urllib.request.urlopen(url).read()
soup = BeautifulSoup(sourcecode, "html.parser")

In [63]:
# Click the options
brand = driver.find_element(By.ID, 'BrandFilter_4')
driver.execute_script("arguments[0].click();", brand)

type = driver.find_element(By.ID, 'TypeFilter_1')
driver.execute_script("arguments[0].click();", type)

category = driver.find_element(By.ID, 'CategoryFilter_1')
driver.execute_script("arguments[0].click();", category)

# Apply the selected options
driver.find_element(By.CSS_SELECTOR, '#filterOverlay > div > div > div > button.btn.btn-primary.btn-lg.btn-ApplyFilter').click()

In [64]:
# Scrape article details
full_address = []

for i in range(5):
    articles = soup.find_all("div", class_="khl-article-block-headline font-weight-bold h4")
    for article in articles:
        address = article.find("a")["href"]
        if address != 'articleurl' and address not in full_address:  # 'articleurl'이 아니고 중복되지 않은 경우에만 추가
            full_address.append(address)
        print(full_address)
        
    next_page_button = driver.find_element(By.CSS_SELECTOR, '#pagination > li.page-item.next > a').click()
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

full_address

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.international-construction.com/news/peri-uses-innovative-approach-on-tunnelling-project/8029215.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.international-construction.com/news/peri-uses-innovative-approach-on-tunnelling-project/8029215.article', 'https://www.international-construction.com/news/interview-caterpillar-executive-on-electric-power-automation-and-new-equipment/8029165.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.international-construction.com/news/peri-uses-innovative-approach-on-tunnelling-project/8029215.article', 'https://www.international-construction.com/news/inte

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article',
 'https://www.international-construction.com/news/peri-uses-innovative-approach-on-tunnelling-project/8029215.article',
 'https://www.international-construction.com/news/interview-caterpillar-executive-on-electric-power-automation-and-new-equipment/8029165.article',
 'https://www.construction-europe.com/news/digging-with-a-twist-what-are-the-advantages-of-a-tiltrotator-/8027719.article',
 'https://www.construction-europe.com/news/triangle-tyre-increases-european-focus/8028851.article',
 'https://www.construction-europe.com/news/digital-construction-site-steals-show-on-samoter-s-first-day/8028738.article',
 'https://www.international-construction.com/news/7-things-we-learned-from-caterpillar-s-latest-trading-update/8028685.article',
 'https://www.international-construction.com/news/north-america-drives-q1-rise-for-caterpillar/8028592.article',
 'https://www.construction-europe.

In [65]:
article_data = []
for article_link in full_address:
    driver.get(article_link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    release_date = ""
    title = ""
    content = ""
   
    # Scrape release date
    release_date_elem = soup.find("span", class_="PubDate")
    release_date = release_date_elem.text.strip() if release_date_elem else ""
            
    # Scrape news title
    title_elem = soup.find("h1", class_="khl-article-page-title")
    title = title_elem.text.strip() if title_elem else ""

    # Scrape article content
    first = soup.find("div", class_="col-12 khl-article-page-standfirst").text
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])

    # Check if content is empty
    if not content:
        continue

    # Append article data to the list
    article_data.append({"Date": release_date, "Title": title, "Content": content})
driver.quit()

In [68]:
# Create DataFrame from article data
df = pd.DataFrame(article_data)
df

,Date,Title,Content
0,30 May 2023,World’s top ten construction OEMs revealed,"The latest Yellow Table, International Constru..."
1,25 May 2023,Peri uses ‘innovative approach’ on tunnelling ...,"On the A26 tunnelling project in Linz, Austria..."
2,24 May 2023,Interview: Caterpillar executive on electric p...,ConExpo is North America’s largest constructio...
3,18 May 2023,Digging with a twist: What are the advantages ...,"Popular in Scandinavia, and growing in other a..."
4,09 May 2023,Triangle Tyre increases European focus,The China-based manufacturer’s current range o...
5,03 May 2023,Digital construction site steals show on Samot...,"\n\n\n\n\nMachine control, remote control, aut..."
6,02 May 2023,7 things we learned from Caterpillar’s latest ...,\n\nLast week saw Caterpillar unveil a strong ...
7,27 April 2023,North America drives Q1 rise for Caterpillar,Caterpillar has announced first-quarter 2023 s...
8,21 April 2023,Simem celebrates 60 years of concrete plants,\nLike many other small and medium manufacturi...
9,20 April 2023,60 years building concrete plants,\n\n\nSimem is a family-owned manufacturer of ...


In [69]:
# Save DataFrame to CSV
df.to_csv("caterpillar_article_final.csv", index=False)